In [ ]:
from online_data.scan_monitor.scan_acquisition import callback
from online_data.scan_monitor.scan_acquisition import initialize_scan_monitor
from online_data.scan_monitor.scan_acquisition import unregister_all

import holoviews as hv
from holoviews import opts
from holoviews import streams
from holoviews.streams import Pipe
from bokeh.plotting import show
hv.extension('bokeh')

import numpy as np

In [ ]:
initialize_scan_monitor()

In [ ]:
unregister_all() #unregisters all callback functions

## example of plotting 1d data from digitizer ##

In [ ]:
"""Plotting 1d data"""

spectrum_data_buffer_example1 = Pipe(data=([], ''))

@callback
def on_data_example1(in_data):
   
    #print('IN DATA: ', in_data)
    
    global spectrum_data_buffer
    global scan_number1
    
    if in_data['type'] == 'data_desc': # Started new scan, got data description
        #print('Started new scan')
        scan_number1 = in_data['data']['serialno']
        return
    
    if in_data['type'] == 'record_end': # End of scan
        #print('Scan ended')
        return
    
    if in_data['type'] == 'record_data': # Record data
        records_dict = in_data['records']
        records_indxs = list(records_dict.keys())
        last_record = records_dict[records_indxs[-1]]
        data_sources = list(last_record.data.keys()) # list of data sources in recordsData
        last_1d_data = last_record.data[data_sources[0]] # taking the last one in the actual used Measurement Group only a digitizer is utilized
        spectrum_data_buffer_example1.send((last_1d_data, str(scan_number1)))
        
        
def spectrum_plot_example1(data):
    
    timeseries_data, scan_number_str = data
    
    return hv.Curve(timeseries_data).opts(xlabel='Time', title=f'Scan number: {scan_number_str}', ylabel='Intensity (mV)', width=900, height=300) #, framewise=True)




In [ ]:
hv.DynamicMap(spectrum_plot_example1, streams=[spectrum_data_buffer_example1])

In [ ]:
#

## Example of plotting an integrated intensity of peak as a function of motor position within a scan ##

Example demonstrate how to plot a data received from scan as a function of position of motor/movable.

In [ ]:
"""Plotting integrated data"""

scan_data_buffer_example2=Pipe(data=([], ''))

integration_data2 = []

movable_name = None

@callback
def on_data_example2(in_data):
    
    global scan_data_buffer2
    global integration_range
    global integration_data2
    
    global movable_name2
    
    global scan_number2
    
    #print('IN DATA: ', in_data)
    
    if in_data['type'] == 'data_desc': # Started new scan, got data description
        #print('Started new scan')
        integration_data2 = []                
        movable_name2 = in_data['data']['ref_moveables'][0]
        
        scan_number2 = in_data['data']['serialno']
        return
    
    if in_data['type'] == 'record_end': # End of scan
        #print('Scan ended')
        return
    
    if in_data['type'] == 'record_data' and movable_name2 is not None: # Record data
        
        #integration_data[0].append(in_data['data'][movable_name])
        records_dict = in_data['records']
        records_indxs = list(records_dict.keys())
        last_record = records_dict[records_indxs[-1]]
        data_sources = list(last_record.data.keys()) # list of data sources in recordsData

        peak_integral = np.sum(last_record.data[data_sources[0]])
        integration_data2.append((in_data['data'][movable_name2], peak_integral))
        scan_data_buffer_example2.send((integration_data2, str(scan_number2)))
        
def spectrum_plot_example2(data):

    timeseries_data, scan_number_str = data
    
    return hv.Curve(timeseries_data).opts(xlabel='motor',title=f'Scan number: {scan_number_str}', ylabel='Intensity (a.u.)', width=900, height=600, framewise=True) * hv.Scatter(timeseries_data).opts(color='black', size=20, marker='o')




In [ ]:
hv.DynamicMap(spectrum_plot_example2, streams=[scan_data_buffer_example2])

In [ ]:
#

## Example of plotting the signal from scans performed manually ##

Example demonstrate how to plot a data received from a scan which is performed manually.

In [ ]:
"""Plotting integrated data"""

scan_data_buffer_example3=Pipe(data=([], ''))

intergation_range = (50,100)
manual_scan_data = [] # x, y


@callback
def on_data_example3(in_data):
    
    global scan_data_buffer_example3
    global scan_data_accu
    global scan_number3
    
    #print('IN DATA: ', in_data)
    
    if in_data['type'] == 'data_desc': # Started new scan, got data description
        #print('Started new scan')
        scan_data_accu = None               
        scan_number3 = in_data['data']['serialno']
        return
    
    if in_data['type'] == 'record_end' and scan_number3 is not None: # End of scan
        peak_integral = np.sum(scan_data_accu)
        #integration_data[1].append(peak_integral)
        manual_scan_data.append((scan_number3, peak_integral))
        scan_data_buffer_example3.send((manual_scan_data, ''))
        return
    
    if in_data['type'] == 'record_data'  and scan_number3 is not None: # Record data
        
        #integration_data[0].append(in_data['data'][movable_name])
        records_dict = in_data['records']
        records_indxs = list(records_dict.keys())
        last_record = records_dict[records_indxs[-1]]
        data_sources = list(last_record.data.keys()) # list of data sources in recordsData
        
        if scan_data_accu is None:
            scan_data_accu = last_record.data[data_sources[0]] # taking the last one in the actual used Measurement Group only a digitizer is utilized
        else:
            scan_data_accu += last_record.data[data_sources[0]]
            
        
def spectrum_plot_example3(data):

    timeseries_data, text = data
    
    return hv.Curve(timeseries_data).opts(xlabel='Scan number', ylabel='Intensity (a.u.)', width=900, height=600, framewise=True) * hv.Scatter(timeseries_data).opts(color='black', size=20, marker='o')



In [ ]:
hv.DynamicMap(spectrum_plot_example3, streams=[scan_data_buffer_example3])

In [ ]:
#